# BERT word embedding + XGBoost
## First attempt

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as functional
import matplotlib.pyplot as plt
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig
import gc
from transformers import BertModel
from sklearn.metrics import roc_auc_score,f1_score
import time
import datetime
from bs4 import BeautifulSoup
import re,string,unicodedata
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

import tensorflow as tf
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
fake = pd.read_csv('/content/drive/MyDrive/thesis-data/Fake.csv')
true = pd.read_csv('/content/drive/MyDrive/thesis-data/True.csv')

In [5]:
fake["label"] = 0
true["label"] = 1

In [6]:
df = pd.concat([fake, true], ignore_index = True)

In [7]:
df['text'] = df['title'] + " " + df['text']
df.drop(columns=['title', 'date', 'subject'], inplace = True)

In [8]:
nltk.download('stopwords')

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
#Apply function on review column
df['text']=df['text'].apply(denoise_text)

<ipython-input-9-4f1d5a4b9299>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [10]:
from transformers import BertTokenizer
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [11]:
df_original = df.copy()
df = df.sample(frac=1).reset_index(drop=True)[:1000]

In [12]:
# texts = df.text.values
# labels = df.label.values

text = df['text']
target = df['label']

In [13]:
def bert_encode(input_text, max_len):
    input_ids = []
    attension_masks = []
    for text in input_text:
        output_dict = tokenizer.encode_plus(
            text, 
            add_special_tokens = True,
            truncation=True,
            max_length = max_len,
            pad_to_max_length = True,
            return_attention_mask = True
        )
        input_ids.append(output_dict['input_ids'])
        attension_masks.append(output_dict['attention_mask'])
    return np.array(input_ids), np.array(attension_masks)

In [21]:
train_input_ids, train_attention_masks = bert_encode(text, 256)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [27]:
# Train an XGBoost model using the BERT embeddings and attention mask features
dtrain = xgb.DMatrix(np.concatenate([train_input_ids[:900], train_attention_masks[:900]], axis=1), label=labels[:900])
dtest = xgb.DMatrix(np.concatenate([train_input_ids[901:], train_attention_masks[901:]], axis=1), label=labels[901:])
params = {
    'objective': 'multi:softmax',
    'num_class': len(np.unique(labels[:900])),
    'eval_metric': 'merror',
    'max_depth': 6
}
model = xgb.train(params, dtrain)

# Make predictions on the testing set and evaluate the model
y_pred = model.predict(dtest)
accuracy = accuracy_score(labels[901:], y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8585858585858586
